In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install learn2learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.7/841.7 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 66.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.9 MB/s eta 0:00:00
  Created wheel for learn2learn: filename=learn2learn-0.1.7-cp37-cp37m-linux_x86_64.whl size=1264403 sha256=4390ceaa51858567a095305ba2780f686cd4a188988d0ed7a9322c6d953c1bde
  Stored in directory: /root/.cache/pip/wheels/66/29/ac/1d46fdb88fb1fb02491123ef3fcec13d5363eb14fec6f8af05
  Created wheel for qpth: filename=qpth-0.0.15-py3-none-any.whl size=15379 sha256=266a5fc9

In [3]:
!git clone https://github.com/parnianrazavipour/Ano-MetaTL.git
%cd Ano-MetaTL
!git checkout origin/ad_maml

[autoreload of google.oauth2.service_account failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/conda/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/opt/conda/lib/python3.7/site-packages/google/oauth2/service_account.py", line 90, in <module>
    credentials.CredentialsWithTokenUri,
AttributeError: module 'google.auth.credentials' has no attribute

Cloning into 'Ano-MetaTL'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 51 (delta 18), reused 37 (delta 10), pack-reused 0
Unpacking objects: 100% (51/51), 2.43 MiB | 7.66 MiB/s, done.
/kaggle/working/Ano-MetaTL
Note: switching to 'origin/ad_maml'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 70e3c29 Kaggle Notebook | notebook642bf01337 | Version 7


In [4]:
# drive.mount('/content/drive')


In [5]:
# import os
# path = "../drive/MyDrive/Rabiee/Social/metatl_beauty/models"
# os.system(f'cp -r {path} .')
# path = "../drive/MyDrive/Rabiee/Social/data/"
# os.system(f'cp -r {path} .')


In [6]:
import os 
import pandas as pd
import numpy as np

k = 5
min_intrac = k+1 
#set w.r.t k
train_test_p = 0.7

ratings = pd.read_csv("/kaggle/input/ratings-beauty/ratings_Beauty.csv",
                      names=['user_id', 'product_id','rating','timestamp'])

ratings = ratings[ratings.groupby('user_id')['user_id'].transform('count').ge(min_intrac)]

start_df = ratings.groupby(['user_id'])['timestamp'].agg('min')
ratings['start_ts'] = ratings['user_id'].apply(lambda x: start_df[x])
ratings = ratings.sort_values(['start_ts', 'user_id', 'timestamp'], ascending=[True, True, True])

all_user, idx = np.unique(ratings['user_id'], return_index=True)
user_dic = {}
all_user = all_user[np.argsort(idx)]
for i, user in enumerate(all_user):
    user_dic[user] = i + 1
ratings['user_id'] =  ratings['user_id'].apply(lambda user: user_dic[user])

all_item, item_idx = np.unique(ratings['product_id'], return_index=True)
item_dic = {}
all_item = all_item[np.argsort(item_idx)]
for i, item in enumerate(all_item):
    item_dic[item] = i + 1
ratings['product_id'] =  ratings['product_id'].apply(lambda item: item_dic[item])

ratings = ratings.reset_index(drop=True)
ratings = ratings.drop(['rating', 'start_ts'], axis=1)

sep_user = max(ratings.user_id) * (train_test_p)
train_df = ratings[ratings['user_id'] <= sep_user]
test_df = ratings[ratings['user_id'] > sep_user]
test_df = test_df[test_df.product_id <= max(train_df.product_id)]

if not os.path.exists('./data'):
    os.mkdir('./data')
if not os.path.exists('./data/beauty'):
    os.mkdir('./data/beauty')
    
train_df.to_csv('data/beauty/beauty_train.csv', sep='\t', header=False, index=False)
test_df.to_csv('data/beauty/beauty_test_new_user.csv', sep='\t', header=False, index=False)

In [7]:
%cd Ano-MetaTL
%pwd
# !conda install pytorch --yes

[Errno 2] No such file or directory: 'Ano-MetaTL'
/kaggle/working/Ano-MetaTL


'/kaggle/working/Ano-MetaTL'

In [8]:
import torch, random
import os 

class Arguments:
    def __init__(self):
        self.dataset = "beauty"
        self.seed = 0
        self.K = 5
        self.embed_dim = 100
        self.batch_size = 1024
        self.learning_rate = 0.001
        self.epoch = 20000
        self.print_epoch = 100
        self.eval_epoch = 500
        self.beta = 5
        self.margin = 1
        self.dropout_p = 0.5
        self.device = 0
        self.ano_k = self.K
        self.adapt_lr = 0.01
        self.meta_lr = 0.005
        self.metatl_path = 'models/metatl.pt'
        self.ad_maml_path = 'models/maml_ad.pt'

args = Arguments()
args.device = torch.device('cuda:'+str(args.device))
params = {}
for attr, value in args.__dict__.items():
    params[attr] = value
# params['device'] = torch.device('cuda:'+str(args.device))
# args.device = 'cpu'
# params['device'] = 'cpu'

In [9]:
from trainer import *
from utils import *
from sampler import *
import json


if params['seed'] is not None:
    SEED = params['seed']
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    np.random.seed(SEED)
    random.seed(SEED)

user_train, usernum_train, itemnum, user_input_test, user_test, user_input_valid, user_valid = data_load(args.dataset, args.K)    

train_dataset = WarpSampler(user_train, usernum_train, itemnum, sample_function_mixed, batch_size=args.batch_size, maxlen=args.K, n_workers=3)

test_dataset = DataLoader(user_input_test, user_test, itemnum, params)

val_dataset = DataLoader(user_input_valid, user_valid, itemnum, params)


user_train, usernum_train, itemnum, user_input_test, user_test, user_input_valid, user_valid = data_load(args.dataset, args.K)    

trainer = Trainer([train_dataset, val_dataset ,test_dataset], itemnum, params)

trainer.train()


Epoch: 19600	Loss: 0.0202Epoch: 19700	Loss: 0.0160Epoch: 19800	Loss: 0.0219Epoch: 19900	Loss: 0.0250Finish


In [10]:
%pwd


'/kaggle/working/Ano-MetaTL'

# New Section

In [11]:
# from torch import nn
# from models import Embedding, MetaLearner
# from ad_maml.model import AnomalyDetector
# from torch.autograd import grad
# from torch import optim
# import learn2learn as l2l
# from collections import OrderedDict


# class AnoMetaTL(nn.Module):
#     def __init__(self, max_item, ad_maml, args):
#         super(AnoMetaTL, self).__init__()
#         self.device = args.device
#         self.beta = args.beta
#         self.embedding = nn.Embedding(max_item + 1, args.embed_dim)
#         nn.init.xavier_uniform_(self.embedding.weight)
#         self.relation_learner = MetaLearner(args.K - 1, embed_size=args.embed_dim, num_hidden1=500,
#                                                     num_hidden2=200, out_size=100, dropout_p=args.dropout_p)
#         self.loss_func = nn.MarginRankingLoss(args.margin)
#         self.ad_maml = ad_maml

#     def split_concat(self, positive, negative):
#         pos_neg_e1 = torch.cat([positive[:, :, 0, :],
#                                 negative[:, :, 0, :]], 1).unsqueeze(2)
#         pos_neg_e2 = torch.cat([positive[:, :, 1, :],
#                                 negative[:, :, 1, :]], 1).unsqueeze(2)
#         return pos_neg_e1, pos_neg_e2

#     def embedding_triples(self, triples):
#         idx = [[[t[0], t[2]] for t in batch] for batch in triples]
#         idx = torch.LongTensor(idx).to(self.device)
#         return self.embedding(idx)

#     def calculate_p_n_score(self, start, target, transition, k):
#         score = -torch.norm(start + transition - target, 2, -1).squeeze(2)
#         p_score = score[:, :k]
#         n_score = score[:, k:]
#         return p_score, n_score
    
#     def get_ano_support_task(self, task):
#         support_triples, support_negative_triples, \
#             query_triples, negative_triples, = [item[0] for item in task]
#         support = [*np.array(support_triples)[:, 0], np.array(support_triples)[-1, -1]]
#         support_neg = [*np.array(support_negative_triples)[:, 2]]
#         return support, support_neg

#     def get_ano_query_task(self, task):
#         support_triples, support_negative_triples, \
#             query_triples, negative_triples, = [item[0] for item in task]
#         support = [*np.array(support_triples)[:, 0], np.array(support_triples)[-1, -1]]
#         query = []
#         query.append([*support[-self.ad_maml.k + 1:], query_triples[0][2]])
#         for neg_triple in negative_triples:
#             query.append([*support[-self.ad_maml.k + 1:], neg_triple[2]])
#         return query

#     def get_ano_preds(self, ad_maml_clone, data):
#         data_embed = self.embedding(data)
#         data_ano = data_embed[-1].unsqueeze(0)
#         data_embed = data_embed.reshape(-1).unsqueeze(0)
#         preds = ad_maml_clone(data_embed, data_ano)
#         return preds.squeeze(0)

#     def ano_support_forward(self, ad_maml_clone, task):
#         ano_k = ad_maml_clone.k
#         support_y = []
#         support_neg_y = []
#         all_support, all_support_neg = self.get_ano_support_task(task)
#         for i, neg in enumerate(all_support_neg[:ano_k - 2]):
#             support = all_support[:ano_k]
#             support[-1], support[i + 1] = support[i + 1], support[-1]
#             support = torch.tensor(support).to(self.device)
#             support_y.append(self.get_ano_preds(ad_maml_clone, support))
#             support_neg = [*support[:-1], neg]
#             support_neg = torch.tensor(support_neg).to(self.device)
#             support_neg_y.append(self.get_ano_preds(ad_maml_clone, support_neg))
#         for i, neg in enumerate(all_support_neg[ano_k - 2:]):
#             support = all_support[i: i + ano_k]
#             support = torch.tensor(support).to(self.device)
#             support_y.append(self.get_ano_preds(ad_maml_clone, support))
#             support_neg = [*support[:-1], neg]
#             support_neg = torch.tensor(support_neg).to(self.device)
#             support_neg_y.append(self.get_ano_preds(ad_maml_clone, support_neg))
#         support_y = torch.cat(support_y)
#         support_neg_y = torch.cat(support_neg_y)
#         return support_y, support_neg_y

#     def ano_query_forward(self, ad_maml_clone, task):
#         query_y = []
#         all_query = self.get_ano_query_task(task)
#         all_query = torch.tensor(all_query).to(self.device)
#         data_embed = self.embedding(all_query)
#         data_ano = data_embed[:, -1, :]
#         data_embed = data_embed.reshape(data_embed.shape[0], -1)
#         preds = ad_maml_clone(data_embed, data_ano)
#         return preds.reshape(-1)

#     # def load_model(self, metatl_path, ad_maml_path):
#     #     #load embedder
#     #     metatl_data = torch.load(metatl_path, map_location=torch.device(self.device))
#     #     self.embedding.load_state_dict(OrderedDict([('weight', metatl_data['embedding.embedding.weight'])]))
#     #     #load relation learner
#     #     rl_name = 'relation_learner'
#     #     all_keys = []
#     #     for key in metatl_data.keys():
#     #         if  rl_name in key:
#     #             all_keys.append(key)

#     #     rl_data = []
#     #     for key in all_keys:
#     #         rl_data.append((key[len(rl_name) + 1:], metatl_data[key]))
#     #     self.relation_learner.load_state_dict(OrderedDict(rl_data))
#     #     #load ad_maml
#     #     ad_maml_data = torch.load(ad_maml_path, map_location=torch.device(self.device))
#     #     self.ad_maml.load_state_dict(ad_maml_data)

#     def forward(self, task, is_eval=False):
#         # transfer task string into embedding
#         support, support_negative, query, negative = [self.embedding_triples(t) for t in task]
 
#         K = support.shape[1]              # num of K
#         num_sn = support_negative.shape[1]  # num of support negative
#         num_q = query.shape[1]              # num of query
#         num_n = negative.shape[1]           # num of query negative

#         rel = self.relation_learner(support)
#         rel.retain_grad()
#         rel_s = rel.expand(-1, K+num_sn, -1, -1)

#         sup_neg_e1, sup_neg_e2 = self.split_concat(support, support_negative)
#         p_score, n_score = self.calculate_p_n_score(sup_neg_e1, sup_neg_e2, rel_s, K)
#         ad_maml_clone = self.ad_maml.clone()
#         support_y, support_neg_y = self.ano_support_forward(ad_maml_clone, task)
#         p_score = support_y * p_score
#         n_score = support_neg_y * n_score

#         y = torch.ones_like(p_score).to(self.device)
#         support_loss = self.loss_func(p_score, n_score, y)
#         ad_maml_clone.adapt(support_loss)
#         rel_q = rel - self.beta * grad(support_loss, rel, retain_graph=True)[0]
        
#         rel_q = rel_q.expand(-1, num_q + num_n, -1, -1)
#         que_neg_e1, que_neg_e2 = self.split_concat(query, negative)  
#         p_score, n_score = self.calculate_p_n_score(que_neg_e1, que_neg_e2, rel_q, num_q)
        
#         # if not is_eval:
#         query_y = self.ano_query_forward(ad_maml_clone, task)
#         p_score = query_y[0] * p_score
#         n_score = query_y[1:] * n_score

#         return p_score, n_score


# device = args.device
# ad_model = AnomalyDetector(args.ano_k, input_embed_size=args.embed_dim, dropout_p=args.dropout_p)
# ad_maml = l2l.algorithms.MetaSGD(ad_model, lr=args.adapt_lr)
# ad_maml.to(device)

# ano_metatl = AnoMetaTL(itemnum, ad_maml, args)
# ano_metatl.to(device)
# # if args.metatl_path is not None:
# #     ano_metatl.load_model(args.metatl_path, args.ad_maml_path)
# #     print('Used pretrained data')
# optimizer = optim.Adam([{'params': ano_metatl.embedding.parameters()},
#                         {'params': ano_metatl.relation_learner.parameters()},
#                         {'params': ano_metatl.ad_maml.parameters(), 'lr': args.meta_lr}],
#                         args.metatl_lr)

In [12]:
# def update_rank(preds, stats):
#     query_idx = len(preds) - 1
#     _, idx = torch.sort(preds, descending=True)
#     idx = idx.cpu().numpy()
#     rank = np.where(idx == query_idx)[0][0] + 1
#     # update stats
#     if rank <= 10:
#         stats['Hits@10'] += 1
#         stats['NDCG@10'] += 1 / np.log2(rank + 1)
#     if rank <= 5:
#         stats['Hits@5'] += 1
#         stats['NDCG@5'] += 1 / np.log2(rank + 1)
#     if rank == 1:
#         stats['Hits@1'] += 1
#         stats['NDCG@1'] += 1 / np.log2(rank + 1)
#     stats['MRR'] += 1.0 / rank

# def eval_model(ano_metatl, eval_dataset):
#     ano_metatl.eval()
#     eval_dataset.curr_tri_idx = 0
#     stats = {'MRR': 0, 'Hits@1': 0, 'Hits@5': 0, 'Hits@10': 0, 'NDCG@1': 0, 'NDCG@5': 0, 'NDCG@10': 0}
#     eval_batch_size = 0

#     while True:
#         eval_task, curr_rel = eval_dataset.next_one_on_eval()
#         if eval_task == 'EOT':
#             break
#         eval_batch_size += 1
#         p_score, n_score = ano_metatl(eval_task, is_eval=True)
#         preds = torch.cat([n_score, p_score], 1).squeeze()
#         update_rank(preds, stats)

#     for key in stats.keys():
#         stats[key] = stats[key] / eval_batch_size

#     print("Results: \tMRR: {:.3f}\tNDCG@10: {:.3f}\tNDCG@5: {:.3f}\tNDCG@1: {:.3f}\tHits@10: {:.3f}\tHits@5: {:.3f}\tHits@1: {:.3f}\r".format(
#             stats['MRR'], stats['NDCG@10'], stats['NDCG@5'], stats['NDCG@1'], stats['Hits@10'], stats['Hits@5'], stats['Hits@1']))

In [13]:
# for iter in range(args.epoch):
#     ano_metatl.train()
#     meta_train_loss = 0
#     train_task, curr_rel = train_dataset.next_batch()
#     batch_size = len(train_task[0])
#     for index in range(batch_size):
#         task = [[train_task[i][index]] for i in range(len(train_task))]
#         p_score, n_score = ano_metatl(task)
#         y = torch.ones_like(p_score).to(device)
#         meta_train_loss += ano_metatl.loss_func(p_score, n_score, y)
#     meta_train_loss = meta_train_loss / batch_size
#     print(f'Iteration: {iter}, Meta Train Loss: {meta_train_loss.item()}')

#     optimizer.zero_grad()
#     meta_train_loss.backward()
#     optimizer.step()

#     if iter % args.eval_epoch == 0:
#         print(f'Validating, Iteration: {iter}')
#         eval_model(ano_metatl, val_dataset)

#         print(f'Testing, Iteration: {iter}')
#         eval_model(ano_metatl, test_dataset)

In [14]:
# torch.save(ano_metatl.state_dict(), self.metatl_path)
# torch.save(ad_model.state_dict(), self.ad_maml_path)